In [2]:
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from bs4 import BeautifulSoup
import requests

In [3]:
round_ = '17'
url = f'https://www.sportsbet.com.au/betting/australian-rules/afl/round-{round_}'
url

'https://www.sportsbet.com.au/betting/australian-rules/afl/round-17'

In [4]:
requests.get(url)

<Response [200]>

In [6]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [9]:
tmp = soup.find_all('script')

In [10]:
game_urls = []
for i in tmp[:9]:
    url_find = re.compile(r'https://www.sportsbet.com.au/betting/australian-rules/afl/(.*?)\-\d{7}')
    game_url = url_find.search(i.text)
    game_urls.append(game_url.group())
game_urls

['https://www.sportsbet.com.au/betting/australian-rules/afl/richmond-v-sydney-7454960',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/western-bulldogs-v-collingwood-7454970',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/brisbane-v-west-coast-7454971',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/greater-western-sydney-v-hawthorn-7454980',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/st-kilda-v-melbourne-7454981',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/port-adelaide-v-gold-coast-7455001',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/geelong-v-north-melbourne-7455010',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/essendon-v-adelaide-7455011',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/fremantle-v-carlton-7455012']

In [13]:
r = re.compile(r'\/(\w*\-?\w*\-?\w*)\-v\-(\w*\-?\w*\-?\w*)\-')
game_list = []
for game in game_urls:
    game_list.append(r.findall(game))

In [15]:
game_list

[[('richmond', 'sydney')],
 [('western-bulldogs', 'collingwood')],
 [('brisbane', 'west-coast')],
 [('greater-western-sydney', 'hawthorn')],
 [('st-kilda', 'melbourne')],
 [('port-adelaide', 'gold-coast')],
 [('geelong', 'north-melbourne')],
 [('essendon', 'adelaide')],
 [('fremantle', 'carlton')]]

In [16]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(700,800)

In [17]:
disposals = ['15 or more',
            '20 or more',
            '25 or more',
            '30 or more']
output = pd.DataFrame({},columns = {'teams','round','Player', 'disposals', 'odds'})

In [18]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        disposal_market = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])[5]')
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,9):
            dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')
            
            dis.click()
            
            tmp = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
            new_row = {'Player': tmp[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':disposals[i - 5],'odds':tmp[-1]}
            output.loc[len(output)] = new_row
            
            for j in range(50):
                try:
                    tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                    new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':disposals[i - 5],'odds':tmp1[-1]}
                    print(tmp1)
                    output.loc[len(output)] = new_row
                except:
                    continue

            dis.click()
            time.sleep(2)
            
    except:
        continue

['Nick Vlastuin', '1.10']
['Shai Bolton', '1.11']
['Dustin Martin', '1.15']
['Trent Cotchin', '1.24']
['Nathan Broad', '1.25']
['Liam Baker', '1.26']
['Jack Ross', '1.33']
['Jack Graham', '1.41']
['Noah Balta', '1.41']
['Toby Nankervis', '1.57']
['Marlion Pickett', '1.90']
['Kamdyn McIntosh', '1.95']
['Nick Blakey', '1.06']
['James Rowbottom', '1.07']
['Jake Lloyd', '1.08']
['Angus Sheldrick', '1.19']
['Justin McInerney', '1.33']
['Isaac Heeney', '1.35']
['Robbie Fox', '1.35']
['Oliver Florent', '1.41']
['Braeden Campbell', '1.49']
['Dane Rampe', '1.55']
['Harry Cunningham', '1.55']
['Tom Papley', '1.68']
['Tom Hickey', '1.94']
['Hayden McLean', '2.95']
['Shai Bolton', '1.48']
['Daniel Rioli', '1.49']
['Dustin Martin', '1.60']
['Nick Vlastuin', '1.69']
['Trent Cotchin', '1.94']
['Liam Baker', '1.97']
['Jack Ross', '2.25']
['Nathan Broad', '2.30']
['Jack Graham', '2.55']
['Noah Balta', '3.00']
['Toby Nankervis', '3.05']
['Kamdyn McIntosh', '5.75']
['Marlion Pickett', '6.50']
['Luke Park

['Noah Long', 'Last 5: N/A', '2.25']
['Jamie Cripps', 'Last 5: 10-6-17-12-13', '2.90']
['Hugh McCluggage', 'Last 5: 34-20-18-16-30', '1.13']
['Will Ashcroft', 'Last 5: N/A', '1.24']
['Conor McKenna', 'Last 5: N/A', '2.25']
['Keidean Coleman', 'Last 5: 23-12-20-6-18', '2.25']
['Darcy Wilmot', 'Last 5: 21-16-19-18-12', '2.55']
['Jarrod Berry', 'Last 5: 16-19-18-15-11', '2.60']
['Zac Bailey', 'Last 5: 12-16-26-19-25', '2.75']
['Joe Daniher', 'Last 5: 19-19-12-10-8', '3.70']
['Jaspa Fletcher', 'Last 5: N/A', '3.75']
['Deven Robertson', 'Last 5: 15-7-15-19-1', '4.20']
['Harris Andrews', 'Last 5: 13-21-7-16-5', '4.25']
['Cam Rayner', 'Last 5: 14-11-14-14-15', '4.30']
['Oscar McInerney', 'Last 5: 9-10-18-8-13', '4.30']
['Brandon Starcevich', 'Last 5: 17-10-4-12-10', '6.25']
['Ryan Lester', 'Last 5: 14-18-11-10-11', '7.00']
['Tim Kelly', 'Last 5: 27-27-29-29-26', '1.17']
['Luke Shuey', 'Last 5: 28-28-16-17-9', '1.28']
['Shannon Hurn', 'Last 5: 30-5-27-22-19', '1.31']
['Dom Sheed', 'Last 5: 25-

['Will Day', 'Last 5: 26-29-27-20-30', '2.75']
['Conor Nash', 'Last 5: 18-24-25-21-27', '4.00']
['Jai Newcombe', 'Last 5: 21-20-28-19-31', '4.00']
['Jack Scrimshaw', 'Last 5: 18-31-19-19-16', '6.25']
['Josh Ward', 'Last 5: 28-15-20-20-20', '7.25']
['Josh Weddle', 'Last 5: N/A', '7.25']
['Karl Amon', 'Last 5: 19-18-20-25-23', '9.00']
['Blake Hardwick', 'Last 5: 12-38-22-25-17', '9.50']
['Seamus Mitchell', 'Last 5: 17-24-21-20-19', '10.00']
['Dylan Moore', 'Last 5: 15-17-25-22-31', '13.00']
['Harry Morrison', 'Last 5: 24-16-24-18-18', '14.00']
['Nasiah Wanganeen-Milera', 'Last 5: 22-24-22-30-25', '1.08']
['Mason Wood', 'Last 5: 24-24-18-16-18', '1.11']
['Rowan Marshall', 'Last 5: 26-13-17-16-17', '1.15']
['Jade Gresham', 'Last 5: 21-11-16-18-15', '1.31']
['Ryan Byrnes', 'Last 5: 12-7-10-19-17', '1.32']
['Marcus Windhager', 'Last 5: 20-12-20-15-4', '1.33']
['Callum Wilkie', 'Last 5: 16-15-17-23-21', '1.43']
['Liam Stocker', 'Last 5: 9-18-13-13-23', '1.49']
['Jimmy Webster', 'Last 5: 15-15

['Ben Ainsworth', 'Last 5: 8-27-22-14-14', '3.40']
['Bailey Humphrey', 'Last 5: N/A', '3.45']
['Charlie Ballard', 'Last 5: 18-12-17-9-14', '4.90']
['Jack Lukosius', 'Last 5: 14-11-18-16-12', '5.80']
['Nick Holman', 'Last 5: 6-10-18-17-12', '6.25']
['Connor Rozee', 'Last 5: 23-25-23-29-27', '1.62']
['Dan Houston', 'Last 5: 32-31-17-20-25', '1.93']
['Ollie Wines', 'Last 5: 13-28-18-24-21', '2.25']
['Dylan Williams', 'Last 5: 22-19-16-19-13', '6.25']
['Jason Horne-Francis', 'Last 5: 10-13-13-18-23', '6.25']
['Kane Farrell', 'Last 5: 25-15-17-14-23', '6.25']
['Travis Boak', 'Last 5: 8-13-26-12-12', '6.50']
['Riley Bonner', 'Last 5: 14-9-2-16-16', '11.00']
['Jeremy Finlayson', 'Last 5: 14-10-8-17-17', '11.50']
['Willem Drew', 'Last 5: 18-14-14-17-10', '12.50']
['Ryan Burton', 'Last 5: 16-13-13-7-10', '14.00']
['Darcy Byrne-Jones', 'Last 5: 8-14-7-12-12', '19.00']
['Noah Anderson', 'Last 5: 35-28-27-28-26', '1.42']
['Touk Miller', 'Last 5: 13-26-26-32-20', '1.72']
['Sam Flanders', 'Last 5: 2

In [33]:
output['away'].unique()

array(['SYD', 'COL', 'WCE', 'HAW', 'MEL', 'GCS'], dtype=object)

In [26]:
output['home'] = output['teams'].apply(lambda x: x.split(' v ')[0])
output['away'] = output['teams'].apply(lambda x: x.split(' v ')[1])

In [27]:
team_mapping = {'richmond':'RCH','geelong':'GEE',
                'kangaroos':'NTH','port-adelaide':'POR',
                'gold-coast':'GCS','melbourne':'MEL',
                'greater-western-sydney':'GWS','hawthorn':'HAW',
                'carlton':'CAR','brisbane':'BRL',
                'collingwood':'COL','adelaide':'ADE',
                'fremantle':'FRE','western-bulldogs':'WBD',
                'sydney':'SYD','west-coast':'WCE',
                'st-kilda':'STK','essendon':'ESS'}

In [31]:
for key in team_mapping.keys():
     output.loc[output['home']==key, 'home'] = team_mapping[key]
for key in team_mapping.keys():
    output.loc[output['away']==key, 'away'] = team_mapping[key]     

In [36]:
output = output.drop(columns='hemr')

In [39]:
output

,round,teams,disposals,odds,Player,home,away
0,R17,richmond v sydney,15 or more,1.07,daniel rioli,RCH,SYD
1,R17,richmond v sydney,15 or more,1.10,nick vlastuin,RCH,SYD
2,R17,richmond v sydney,15 or more,1.11,shai bolton,RCH,SYD
3,R17,richmond v sydney,15 or more,1.15,dustin martin,RCH,SYD
4,R17,richmond v sydney,15 or more,1.24,trent cotchin,RCH,SYD
...,...,...,...,...,...,...,...
602,R17,port-adelaide v gold-coast,30 or more,7.50,matt rowell,POR,GCS
603,R17,port-adelaide v gold-coast,30 or more,10.00,darcy macpherson,POR,GCS
604,R17,port-adelaide v gold-coast,30 or more,14.00,lachie weller,POR,GCS
605,R17,port-adelaide v gold-coast,30 or more,15.00,wil powell,POR,GCS


In [40]:
output.to_csv('/Users/caseyh/Desktop/footyfootyfooty/test_disposal_markets_r17.csv', index = False)